In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
import seaborn as sns
from scipy import stats
from pathlib import Path
import pickle
import numpy as np

In [2]:
# display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### I tried to open the whole leavetimes csv and split it but there's always memory error
maybe too many people are working on the server, so in this file, I ll try to clean and merge data with chunksize first

In [3]:
cd ~/tmp/data

/home/team9/tmp/data


In [4]:
ls

Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
leavetimes/
rt_leavetimes_DB_2018.txt
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
rt_vehicles_DB_2018.zip
trips_modelling.csv
weather_factorised.csv
weather_trips.csv


## get unique tripid with lineid with direction

In [5]:
trips = pd.read_csv('rt_trips_DB_2018.txt', sep=";", parse_dates=[1])

In [6]:
trips

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,2018-02-07,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,2018-02-07,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,2018-02-07,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,2018-02-07,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,2018-02-07,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182632,DB,2018-05-14,6765849,123,123_36,2,61560,57840,61365.0,57859.0,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",3216350,"
2182633,DB,2018-05-14,6765469,75,75_17,1,53416,48600,NaN,48823.0,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",2865284,"
2182634,DB,2018-05-14,6765486,33D,33D_62,2,29460,26400,29904.0,NaN,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",3077688,"
2182635,DB,2018-05-14,6764987,70,70_60,1,65277,60600,66341.0,NaN,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",3208841,"


In [7]:
# get unique tripid with lineid with direction
def select_trip(x):
    x = x.drop_duplicates(subset=['TRIPID','LINEID','DIRECTION'],keep='first')
    x = x[['TRIPID','LINEID','DIRECTION']]
    return x

In [8]:
trips = select_trip(trips)

In [9]:
trips.nunique()

TRIPID       658964
LINEID          130
DIRECTION         2
dtype: int64

## read leavetimes in chunksize 

In [10]:
# Read column names from file
cols = list(pd.read_csv("rt_leavetimes_DB_2018.txt", nrows =1))
print(cols)

['DATASOURCE;DAYOFSERVICE;TRIPID;PROGRNUMBER;STOPPOINTID;PLANNEDTIME_ARR;PLANNEDTIME_DEP;ACTUALTIME_ARR;ACTUALTIME_DEP;VEHICLEID;PASSENGERS;PASSENGERSIN;PASSENGERSOUT;DISTANCE;SUPPRESSED;JUSTIFICATIONID;LASTUPDATE;NOTE']


In [11]:
leavetimes = pd.read_csv('rt_leavetimes_DB_2018.txt', sep=";",parse_dates=[1], iterator=True,chunksize=5000000)

In [12]:
leavetimes = leavetimes.get_chunk(5000000)

In [13]:
leavetimes

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
1,DB,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
2,DB,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
3,DB,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
4,DB,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,DB,2018-01-17,6118378,3,1283,31000,31000,31135,31147,1000698,NaN,NaN,NaN,NaN,NaN,NaN,25-JAN-18 18:02:03,NaN
4999996,DB,2018-01-17,6112763,2,4521,32634,32634,32802,32802,1000696,NaN,NaN,NaN,NaN,NaN,NaN,25-JAN-18 18:02:03,NaN
4999997,DB,2018-01-17,6118378,2,4521,30834,30834,30912,30956,1000698,NaN,NaN,NaN,NaN,NaN,NaN,25-JAN-18 18:02:03,NaN
4999998,DB,2018-01-17,6118378,1,7564,30600,30600,30566,30566,1000698,NaN,NaN,NaN,NaN,NaN,NaN,25-JAN-18 18:02:03,NaN


In [14]:
leavetimes.nunique()

DATASOURCE             1
DAYOFSERVICE          17
TRIPID             43643
PROGRNUMBER          103
STOPPOINTID         4709
PLANNEDTIME_ARR    67129
PLANNEDTIME_DEP    67129
ACTUALTIME_ARR     70643
ACTUALTIME_DEP     70654
VEHICLEID            971
PASSENGERS             0
PASSENGERSIN           0
PASSENGERSOUT          0
DISTANCE               0
SUPPRESSED             2
JUSTIFICATIONID     1062
LASTUPDATE            17
NOTE                   0
dtype: int64

In [15]:
leavetimes['SUPPRESSED'].value_counts()

0.0    7481
1.0    3763
Name: SUPPRESSED, dtype: int64

In [16]:
leavetimes['SUPPRESSED'].isnull().sum()

4988756

In [17]:
leavetimes['JUSTIFICATIONID'].isnull().sum()

4988756

### drop the columns which always null or contains too many null values， drop constant column datasource


In [18]:
#leavetimes.drop(columns=['PASSENGERS','PASSENGERSIN','PASSENGERSOUT','DISTANCE','SUPPRESSED','JUSTIFICATIONID','LASTUPDATE','NOTE'],inplace=True)

In [19]:
def select_leavetimes(x):
    x.drop(columns=['DATASOURCE','PASSENGERS','PASSENGERSIN','PASSENGERSOUT','DISTANCE','SUPPRESSED','JUSTIFICATIONID','LASTUPDATE','NOTE'],inplace=True)

In [20]:
select_leavetimes(leavetimes)

In [21]:
leavetimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211
1,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267
2,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263
3,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284
4,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209
...,...,...,...,...,...,...,...,...,...
4999995,2018-01-17,6118378,3,1283,31000,31000,31135,31147,1000698
4999996,2018-01-17,6112763,2,4521,32634,32634,32802,32802,1000696
4999997,2018-01-17,6118378,2,4521,30834,30834,30912,30956,1000698
4999998,2018-01-17,6118378,1,7564,30600,30600,30566,30566,1000698


In [22]:
leavetimes.columns.values.tolist()

['DAYOFSERVICE',
 'TRIPID',
 'PROGRNUMBER',
 'STOPPOINTID',
 'PLANNEDTIME_ARR',
 'PLANNEDTIME_DEP',
 'ACTUALTIME_ARR',
 'ACTUALTIME_DEP',
 'VEHICLEID']

## Merge the tripid

In [23]:
leavetimes = pd.merge(leavetimes,trips,how='left')

In [24]:
leavetimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,DIRECTION
0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,1,1
1,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267,1,1
2,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263,1,1
3,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284,1,1
4,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209,1,1
...,...,...,...,...,...,...,...,...,...,...,...
4999995,2018-01-17,6118378,3,1283,31000,31000,31135,31147,1000698,65B,1
4999996,2018-01-17,6112763,2,4521,32634,32634,32802,32802,1000696,65B,1
4999997,2018-01-17,6118378,2,4521,30834,30834,30912,30956,1000698,65B,1
4999998,2018-01-17,6118378,1,7564,30600,30600,30566,30566,1000698,65B,1


In [25]:
leavetimes['LINE_DIRECTION'] = leavetimes['LINEID'] + '_' + leavetimes['DIRECTION'].astype(str)

In [26]:
leavetimes.nunique()

DAYOFSERVICE          17
TRIPID             43643
PROGRNUMBER          103
STOPPOINTID         4709
PLANNEDTIME_ARR    67129
PLANNEDTIME_DEP    67129
ACTUALTIME_ARR     70643
ACTUALTIME_DEP     70654
VEHICLEID            971
LINEID               125
DIRECTION              2
LINE_DIRECTION       244
dtype: int64

In [27]:
leavetimes['LINEID'].isnull().sum()

0

In [28]:
leavetimes['DIRECTION'].isnull().sum()

0

In [29]:
leavetimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,DIRECTION,LINE_DIRECTION
0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,1,1,1_1
1,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267,1,1,1_1
2,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263,1,1,1_1
3,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284,1,1,1_1
4,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209,1,1,1_1
...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2018-01-17,6118378,3,1283,31000,31000,31135,31147,1000698,65B,1,65B_1
4999996,2018-01-17,6112763,2,4521,32634,32634,32802,32802,1000696,65B,1,65B_1
4999997,2018-01-17,6118378,2,4521,30834,30834,30912,30956,1000698,65B,1,65B_1
4999998,2018-01-17,6118378,1,7564,30600,30600,30566,30566,1000698,65B,1,65B_1


In [30]:
leavetimes[leavetimes.LINEID=='27']

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,DIRECTION,LINE_DIRECTION
56411,2018-01-01,5956632,42,4495,65955,65955,65554,65684,2172288,27,1,27_1
56412,2018-01-01,5956630,42,4495,54031,54031,53831,53831,2172288,27,1,27_1
56413,2018-01-01,5962312,42,4495,52831,52831,52720,52745,2172247,27,1,27_1
56414,2018-01-01,5962310,42,4495,40231,40231,40086,40103,2172247,27,1,27_1
56415,2018-01-01,5961854,42,4495,50431,50431,50247,50280,2172251,27,1,27_1
...,...,...,...,...,...,...,...,...,...,...,...,...
4995320,2018-01-17,6117636,1,2353,54000,54000,54611,54611,2172248,27,2,27_2
4995321,2018-01-17,6121136,1,2353,51600,51600,51828,51828,1932366,27,2,27_2
4995322,2018-01-17,6115954,1,2353,53400,53400,53758,53758,2172249,27,2,27_2
4995323,2018-01-17,6117368,1,2353,56400,56400,56431,56431,2172293,27,2,27_2


In [31]:
line=leavetimes.LINEID.unique()
print(line)

['1' '13' '37' '104' '11' '102' '151' '41' '65' '7A' '140' '84' '79A'
 '15A' '17' '44' '40B' '41C' '83A' '238' '25' '38A' '63' '4' '66A' '31'
 '150' '17A' '25B' '31A' '33' '27B' '32' '14' '67' '9' '61' '43' '130'
 '68' '15' '123' '122' '120' '47' '69' '39A' '76' '40' '70' '29A' '25A'
 '38' '49' '65B' '39' '45A' '42' '77A' '27' '54A' '46A' '27A' '40D' '7'
 '53' '33B' '75' '270' '18' '16' '41B' '145' '83' '79' '16C' '66' '15B'
 '185' '26' '56A' '33A' '14C' '59' '220' '184' '142' '114' '76A' '31B'
 '68A' '66B' '111' '236' '84A' '67X' '84X' '239' '42D' '66X' '116' '41X'
 '46E' '38B' '32X' '161' '7B' '118' '25X' '38D' '7D' '44B' '33X' '25D'
 '51D' '15D' '31D' '39X' '27X' '51X' '77X' '69X' '41A' '70D' '68X']


### use 39A file as a test

In [32]:
cd ~/tmp/data/leavetimes

/home/team9/tmp/data/leavetimes


In [33]:
test_39A = pd.read_csv('leavetimes_39A.csv', sep=",")

In [34]:
# sort by tripid
test_39A = test_39A.sort_values(by=['DAYOFSERVICE','TRIPID','PROGRNUMBER'])

In [35]:
test_39A = test_39A.reset_index(drop=True)

### add journey time and delete outlier and null

In [36]:
test_39A.head(100)

,Unnamed: 0,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID
0,133853,2018-01-01,5959380,1,767,32400,32400,32323,32323,2172316,39A
1,134113,2018-01-01,5959380,2,768,32503,32503,32461,32461,2172316,39A
2,134140,2018-01-01,5959380,3,769,32527,32527,32481,32481,2172316,39A
3,134168,2018-01-01,5959380,4,770,32565,32565,32510,32510,2172316,39A
4,134430,2018-01-01,5959380,5,771,32581,32581,32520,32520,2172316,39A
5,134457,2018-01-01,5959380,6,772,32613,32613,32548,32548,2172316,39A
6,134722,2018-01-01,5959380,7,773,32644,32644,32562,32562,2172316,39A
7,134751,2018-01-01,5959380,8,774,32680,32680,32578,32578,2172316,39A
8,134778,2018-01-01,5959380,9,775,32729,32729,32612,32623,2172316,39A
9,135048,2018-01-01,5959380,10,776,32762,32762,32655,32655,2172316,39A


### connect with the previous stoppinid and arrivial time

In [37]:
test_39A['PRESTOPID']=test_39A['STOPPOINTID']

In [38]:
test_39A['PRESTOPID']=test_39A['PRESTOPID'].shift(1)

In [39]:
test_39A.head(50)

,Unnamed: 0,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,PRESTOPID
0,133853,2018-01-01,5959380,1,767,32400,32400,32323,32323,2172316,39A,NaN
1,134113,2018-01-01,5959380,2,768,32503,32503,32461,32461,2172316,39A,767.0
2,134140,2018-01-01,5959380,3,769,32527,32527,32481,32481,2172316,39A,768.0
3,134168,2018-01-01,5959380,4,770,32565,32565,32510,32510,2172316,39A,769.0
4,134430,2018-01-01,5959380,5,771,32581,32581,32520,32520,2172316,39A,770.0
5,134457,2018-01-01,5959380,6,772,32613,32613,32548,32548,2172316,39A,771.0
6,134722,2018-01-01,5959380,7,773,32644,32644,32562,32562,2172316,39A,772.0
7,134751,2018-01-01,5959380,8,774,32680,32680,32578,32578,2172316,39A,773.0
8,134778,2018-01-01,5959380,9,775,32729,32729,32612,32623,2172316,39A,774.0
9,135048,2018-01-01,5959380,10,776,32762,32762,32655,32655,2172316,39A,775.0


In [40]:
test_39A['PRE_PLANNEDTIME_ARR']=test_39A['PLANNEDTIME_ARR']
test_39A['PRE_PLANNEDTIME_ARR']=test_39A['PRE_PLANNEDTIME_ARR'].shift(1)

In [41]:
test_39A['PRE_ACTUALTIME_ARR']=test_39A['ACTUALTIME_ARR']
test_39A['PRE_ACTUALTIME_ARR']=test_39A['PRE_ACTUALTIME_ARR'].shift(1)

In [42]:
test_39A['PLANNED_JOURNEY_TIME']=test_39A['PLANNEDTIME_ARR']-test_39A['PRE_PLANNEDTIME_ARR']

In [43]:
test_39A['ACTUAL_JOURNEY_TIME']=test_39A['ACTUALTIME_ARR']-test_39A['PRE_ACTUALTIME_ARR']

In [44]:
# test_39A=  test_39A.drop(index = test_39A[(test_39A.PLANNED_JOURNEY_TIME <=0)].index.tolist())
# test_39A=  test_39A.drop(index = test_39A[(test_39A.ACTUAL_JOURNEY_TIME <=0)].index.tolist())

In [45]:
test_39A = test_39A.loc[test_39A['PROGRNUMBER'] != 1]

In [46]:
test_39A

,Unnamed: 0,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,PRESTOPID,PRE_PLANNEDTIME_ARR,PRE_ACTUALTIME_ARR,PLANNED_JOURNEY_TIME,ACTUAL_JOURNEY_TIME
1,134113,2018-01-01,5959380,2,768,32503,32503,32461,32461,2172316,39A,767.0,32400.0,32323.0,103.0,138.0
2,134140,2018-01-01,5959380,3,769,32527,32527,32481,32481,2172316,39A,768.0,32503.0,32461.0,24.0,20.0
3,134168,2018-01-01,5959380,4,770,32565,32565,32510,32510,2172316,39A,769.0,32527.0,32481.0,38.0,29.0
4,134430,2018-01-01,5959380,5,771,32581,32581,32520,32520,2172316,39A,770.0,32565.0,32510.0,16.0,10.0
5,134457,2018-01-01,5959380,6,772,32613,32613,32548,32548,2172316,39A,771.0,32581.0,32520.0,32.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809921,57030779,2018-06-28,7115833,56,749,35461,35461,36694,36716,2693197,39A,748.0,35373.0,36607.0,88.0,87.0
1809922,57035272,2018-06-28,7115833,57,750,35523,35523,36784,36803,2693197,39A,749.0,35461.0,36694.0,62.0,90.0
1809923,57039724,2018-06-28,7115833,58,751,35584,35584,36899,36899,2693197,39A,750.0,35523.0,36784.0,61.0,115.0
1809924,57044117,2018-06-28,7115833,59,752,35660,35660,36921,36936,2693197,39A,751.0,35584.0,36899.0,76.0,22.0
